In [71]:
import pandas as pd
import math
class Lista():
    def __init__(self, k):
        self.lista = []
        self.num_neighbours = 5

    def add_pivo(self, pivo):
        self.pivo = pivo
    
    def adiciona(self, new):
        distance = new.calculate_distance(self.pivo)

        if len(self.lista) != 0 and distance > self.lista[-1].calculate_distance(self.pivo):
            return 
        if len(self.lista) == self.num_neighbours:
            self.lista.pop(-1)
        self.lista.append(new)
        self.lista.sort(key=lambda x: x.calculate_distance(self.pivo))

    def qtd_exemplares_por_especie(self):
        epe = {}
        for i in self.lista:
            if epe.get(i.specie) == None:
                epe[i.specie] = 1
            else:
                epe[i.specie] += 1
        return epe
        

    def define_specie(self):
        qtd_por_especie = self.qtd_exemplares_por_especie()
        #print("qtd_por_especie:", qtd_por_especie)
        maior = list(qtd_por_especie.keys())[0]

        for e in qtd_por_especie:
            if qtd_por_especie[e] > qtd_por_especie[maior]:
                maior = e
            elif  qtd_por_especie[e] == qtd_por_especie[maior]:
                maior = self.melhor_media(e, maior)  
        self.pivo.set_specie(maior)
        return True

    def melhor_media(self, especie1, especie2):
        menor = self.lista[0]
        menor_distancia = self.pivo.calculate_distance(menor)
        for i in self.lista:
            distancia_i = self.pivo.calculate_distance(i)
            if (i.specie == especie1 or i.specie == especie2) and distancia_i < menor_distancia:
                menor = i
                menor_distancia = distancia_i
        return menor.specie
        

class Node ():
    def __init__(self, sepalLengthCm, sepalWidthCm, petalLengthCm, petalWidthCm):
        self.sepalLengthCm = sepalLengthCm
        self.sepalWidthCm = sepalWidthCm
        self.petalLengthCm = petalLengthCm
        self.petalWidthCm = petalWidthCm
        
    def set_specie(self, specie):
        self.specie = specie
    
    def calculate_distance(self, other_node):
        sl = self.sepalLengthCm - other_node.sepalLengthCm
        sw = self.sepalWidthCm - other_node.sepalWidthCm
        pl = self.petalLengthCm - other_node.petalLengthCm
        pw = self.petalWidthCm - other_node.petalWidthCm
        
        sl = sl*sl
        sw = sw*sw
        pl = pl*pl
        pw = pw*pw
        
        return math.sqrt(sl+sw+pl+pw)
    
    def show(self):
        print("sepalLengthCm:", self.sepalLengthCm,
        "sepalWidthCm:", self.sepalWidthCm,
        "petalLengthCm:", self.petalLengthCm,
        "petalWidthCm:", self.petalWidthCm,
        "specie:", self.specie)
        

class KNN ():
    def __init__(self, k, tamanho_treino):
        self.k = k
        self.tamanho_treino = tamanho_treino
        self.distance_matrix = []
        self.nodes = []
        self.training_population = None
        self.test_population = None

    def main (self):
        self.set_populations(self.tamanho_treino)
        self.training_algorithm()
        self.test_algorithm()
    
    def set_populations(self, training_population_size):
        iris = pd.read_csv("iris.csv")
        iris = iris.sample(frac=1).reset_index(drop=True)
        self.training_population = iris.iloc[:training_population_size]
        self.test_population = iris.iloc[training_population_size:]
        
    def training_algorithm(self):
        self.nodes = self.make_nodes(self.training_population)
        
    def make_nodes(self, df):
        new_nodes = []
        for index, row in df.iterrows():
            new_node = Node(df["SepalLengthCm"][index],  df["SepalWidthCm"][index],  df["PetalLengthCm"][index],  df["PetalWidthCm"][index])
            new_node.set_specie(df["Species"][index])
            new_nodes.append(new_node)
        return new_nodes        

    def test_algorithm(self):
        test_nodes = self.make_nodes(self.test_population)
        i = 0
        for node in test_nodes:
            minha_lista = Lista(self.k)

            minha_lista.add_pivo(node)
            for node in self.nodes:
                minha_lista.adiciona(node)
            minha_lista.define_specie()
            
            if minha_lista:
                self.nodes.append(minha_lista.pivo)
            if minha_lista.pivo.specie != self.test_population.get(["Species"]).iloc[i,0]:
                print("DIFERENTE")
            i += 1
        

if __name__ == "__main__":
    knn = KNN(7, 70)
    knn.main()

DIFERENTE
DIFERENTE
DIFERENTE
DIFERENTE
DIFERENTE
